In [1]:

import pandas as pd

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [5]:
# Astra DB credentials
ASTRA_DB_CLIENT_ID = "vshpOYJHmbNURkZzrdggKXGo"  
ASTRA_DB_CLIENT_SECRET = "j1F,zphmnR2kZXYFZa-6NCU_seRaAR4-ktZIYqBrsOk3Yh20ayD0QA1Zwlf6Z.8QIyUMtdP8aZcpGHOZw9nOI9Nc_7j1vw81K2J1f3oCF_yLBEBrLmvi4gN573QHhN38"  
SECURE_CONNECT_BUNDLE = r"C:\Users\Admin\Downloads\secure-connect-social-media-data.zip"  

In [6]:
# Connection to Astra DB
auth_provider = PlainTextAuthProvider(ASTRA_DB_CLIENT_ID, ASTRA_DB_CLIENT_SECRET)
cluster = Cluster(cloud={'secure_connect_bundle': SECURE_CONNECT_BUNDLE}, auth_provider=auth_provider)
session = cluster.connect()

In [7]:

keyspace = "default_keyspace"  
session.set_keyspace(keyspace)

In [8]:

insert_query = """
    INSERT INTO post_metrics (post_id, comments,likes, post_type, shares)
    VALUES (%s, %s, %s, %s, %s)
"""

In [10]:
import csv

In [12]:
from cassandra.query import BatchStatement

In [15]:
from uuid import uuid4

In [16]:

csv_file_path = r"C:\Users\Admin\Desktop\InsightFlow\mock_engagement_data.csv"  
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    
   
    batch = BatchStatement()

    
    for row in csv_reader:
        
        post_id = uuid4()
        
       
        post_type = row['post_type']
        likes = int(row['likes'])
        shares = int(row['shares'])
        comments = int(row['comments'])
        
        
        batch.add(insert_query, (post_id, comments, likes, post_type, shares ))

    
    session.execute(batch)
print("Data inserted successfully")

Data inserted successfully


In [17]:
query = "SELECT post_type, likes, shares, comments FROM post_metrics"
rows = session.execute(query)

In [20]:
data = pd.DataFrame(rows, columns=['post_type', 'likes', 'shares', 'comments'])

In [21]:
average_metrics = data.groupby('post_type')[['likes', 'shares', 'comments']].mean().reset_index()

In [22]:
print(average_metrics)

      post_type       likes      shares    comments
0      Carousel  347.387195   74.457317   29.929878
1         Reels  742.471299  197.262840  124.450151
2  Static_Image  200.014663   34.956012   14.941349


In [ ]:
# Closing the connection
cluster.shutdown()